# Tools for NLP

There are lots of feature transformations that need to be done on text data to get it to a point that machine learning algorithms can understand. Luckily, Spark has placed the most important ones in convienent Feature Transformer calls. 

Let's go over them before jumping into the project.

In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()


'C:\\spark-3.3.0-bin-hadoop2'

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

## Tokenizer
<p><a href="http://en.wikipedia.org/wiki/Lexical_analysis#Tokenization">Tokenization</a> is the process of taking text (such as a sentence) and breaking it into individual terms (usually words).  A simple <a href="api/scala/index.html#org.apache.spark.ml.feature.Tokenizer">Tokenizer</a> class provides this functionality.  The example below shows how to split sentences into sequences of words.</p>

<p><a href="api/scala/index.html#org.apache.spark.ml.feature.RegexTokenizer">RegexTokenizer</a> allows more
 advanced tokenization based on regular expression (regex) matching.
 By default, the parameter &#8220;pattern&#8221; (regex, default: <code>"\\s+"</code>) is used as delimiters to split the input text.
 Alternatively, users can set parameter &#8220;gaps&#8221; to false indicating the regex &#8220;pattern&#8221; denotes
 &#8220;tokens&#8221; rather than splitting gaps, and find all matching occurrences as the tokenization result.</p>

In [4]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [5]:
sentenceDataFrame = spark.createDataFrame([
    (0, "Hi I heard about Spark"),
    (1, "I wish Java could use case classes"),
    (2, "Logistic,regression,models,are,neat")
], ["id", "sentence"])

In [7]:
sentenceDataFrame.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish Java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [8]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [10]:
regex_tokenizer = RegexTokenizer(inputCol='sentence', outputCol= 'words', pattern='\\W')

In [11]:
count_tokens = udf(lambda words: len(words), IntegerType())

In [13]:
tokenized = tokenizer.transform(sentenceDataFrame)

In [16]:
tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish Java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [17]:
rg_tokenized = regex_tokenizer.transform(sentenceDataFrame)

In [18]:
rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish Java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



In [19]:
from pyspark.ml.feature import StopWordsRemover

In [24]:
send_def = spark.createDataFrame([
    (0, ['I', 'saw', 'the', 'green', 'horse']),
    (1, ['Mary', 'had', 'a', 'little', 'lamb'])
],['id', 'tokens'] )

In [22]:
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')

In [25]:
remover.transform(send_def).show()

+---+--------------------+--------------------+
| id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, the, gre...| [saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



In [26]:
#n-gram
from pyspark.ml.feature import NGram

In [27]:
wordDataFrame = spark.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])
], ["id", "words"])

In [31]:
ngram = NGram(n=3, inputCol='words', outputCol='grams')

In [32]:
ngram.transform(wordDataFrame).select('grams').show(truncate=False)

+--------------------------------------------------------------------------------+
|grams                                                                           |
+--------------------------------------------------------------------------------+
|[Hi I heard, I heard about, heard about Spark]                                  |
|[I wish Java, wish Java could, Java could use, could use case, use case classes]|
|[Logistic regression models, regression models are, models are neat]            |
+--------------------------------------------------------------------------------+



In [33]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [47]:
testdata = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic,regression,models,are,neat")
], ["label", "sentence"])

In [48]:
testdata.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish Java could...|
|  1.0|Logistic,regressi...|
+-----+--------------------+



In [49]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [50]:
words_data = tokenizer.transform(testdata)

In [51]:
words_data.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|Hi I heard about ...|[hi, i, heard, ab...|
|  0.0|I wish Java could...|[i, wish, java, c...|
|  1.0|Logistic,regressi...|[logistic,regress...|
+-----+--------------------+--------------------+



In [52]:
hasing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')

In [53]:
featurized_data = hasing_tf.transform(words_data)

In [54]:
idf = IDF(inputCol='rawFeatures', outputCol='features')

In [55]:
idf_model = idf.fit(featurized_data)

In [56]:
rescaled_data = idf_model.transform(featurized_data)

In [59]:
rescaled_data.select('label', 'features').show(truncate = False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                      |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(262144,[18700,19036,33808,66273,173558],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                   |
|0.0  |(262144,[19036,20719,55551,58672,98717,109547,192310],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
|1.0 

In [60]:
from pyspark.ml.feature import CountVectorizer

In [61]:
df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

In [62]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [63]:
cv = CountVectorizer(inputCol= 'words', outputCol='features', vocabSize = 3, minDF = 2.0)

In [64]:
model = cv.fit(df)

In [65]:
result = model.transform(df)

In [66]:
result.show(truncate = False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

